<a href="https://colab.research.google.com/github/hrtywhy/Machine-Learning/blob/main/XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
 #%tensorflow_version 2.x
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda, Flatten
from keras.layers import Convolution1D, Dense, Dropout, MaxPooling1D, LSTM
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from keras import callbacks
from keras.callbacks import CSVLogger
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
import numpy as np
np.random.seed(0)

In [4]:
#loading dataset
import pandas as pd
x = pd.read_csv('/content/drive/My Drive/Dataset/skripsi/dos_dataset/clean_2.csv')
y = pd.read_csv('/content/drive/My Drive/Dataset/skripsi/dos_dataset/y.csv')
y=y['Label']
x=x.drop(['Label'], axis=1)

#normalizing the data
normalized_df=(x-x.mean())/x.std()

normalized_df=normalized_df.drop(['Unnamed: 0'], axis=1)
normalized_df=normalized_df.drop(['Unnamed: 0.1'], axis=1)
normalized_df=normalized_df.drop(['Unnamed: 0.1.1'], axis=1)
normalized_df=normalized_df.drop(['Flow Bytes/s'], axis=1)
normalized_df=normalized_df.drop([' Flow Packets/s'], axis=1)

y = pd.get_dummies(y,prefix=['Label'], drop_first=True)

normalized_df.head()

,Source IP,Source Port,Destination IP,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,SYN Flag Count,RST Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Inbound
0,-0.030358,1.395596,0.157586,-0.607399,-2.031188,-0.191426,0.003253,0.039938,-0.725421,-0.006304,-0.954992,-0.935156,-0.945029,-0.147931,-0.050579,0.342849,0.015085,-0.076871,-0.178761,-0.175445,-0.174509,-0.007595,-0.190368,-0.178330,-0.168507,-0.173212,-0.007115,-0.119406,-0.137760,-0.138345,-0.132072,0.461697,-0.090471,0.187873,0.004551,-1.335866,-0.070783,-0.935040,-0.940755,-0.948545,-0.137098,-0.069737,-0.015726,-0.090471,2.209740,-0.153371,-0.101473,-0.245062,-0.933777,-0.945029,0.015085,0.187873,0.003253,-0.725421,0.039938,-0.006304,-0.054584,-0.057299,0.066885,0.193064,-0.06319,-0.067103,-0.073003,-0.033187,-0.199447,-0.162793,-0.207939,-0.181785,0.19758
1,-0.030358,0.347887,0.157586,-1.625958,-2.031188,-0.209371,-0.007162,0.039938,-0.733744,-0.006304,-0.954992,-0.935156,-0.945029,-0.147931,-0.050579,0.342849,0.015085,-0.076871,-0.216151,-0.222876,-0.212037,-0.007595,-0.208347,-0.227006,-0.222546,-0.210834,-0.007115,-0.119406,-0.137760,-0.138345,-0.132072,0.461697,-0.090471,0.187873,0.004551,-1.324001,0.465069,-0.935040,-0.940755,-0.948545,-0.137098,-0.069737,-0.015726,-0.090471,2.209740,-0.153371,-0.101473,2.722226,-0.933085,-0.945029,0.015085,0.187873,-0.007162,-0.733744,0.039938,-0.006304,7.027636,-0.057299,-0.043324,0.193064,-0.06319,-0.067103,-0.073003,-0.033187,-0.199447,-0.162793,-0.207939,-0.181785,0.19758
2,-0.030358,1.196809,0.157586,-0.560525,-2.031188,-0.193320,0.003253,0.039938,-0.725421,-0.006304,-0.954992,-0.935156,-0.945029,-0.147931,-0.050579,0.342849,0.015085,-0.076871,-0.182709,-0.180450,-0.178468,-0.007595,-0.192263,-0.183461,-0.174204,-0.177179,-0.007115,-0.119406,-0.137760,-0.138345,-0.132072,0.461697,-0.090471,0.187873,0.004551,-1.335864,-0.070722,-0.935040,-0.940755,-0.948545,-0.137098,-0.069737,-0.015726,-0.090471,2.209740,-0.153371,-0.101473,-0.245062,-0.933777,-0.945029,0.015085,0.187873,0.003253,-0.725421,0.039938,-0.006304,-0.054584,-0.057299,0.066885,0.193064,-0.06319,-0.067103,-0.073003,-0.033187,-0.199447,-0.162793,-0.207939,-0.181785,0.19758
3,-0.030358,0.892029,0.157586,-1.562081,-2.031188,-0.194976,0.003253,0.039938,-0.725421,-0.006304,-0.954992,-0.935156,-0.945029,-0.147931,-0.050579,0.342849,0.015085,-0.076871,-0.186161,-0.184840,-0.181939,-0.007586,-0.193921,-0.187948,-0.179186,-0.180647,-0.007107,-0.119405,-0.137758,-0.138345,-0.132072,0.746914,-0.090471,0.187873,0.004551,-1.335861,-0.070655,-0.935040,-0.940755,-0.948545,-0.137098,-0.069737,-0.015726,-0.090471,2.209740,-0.153371,-0.101473,-0.245062,-0.933777,-0.945029,0.015085,0.187873,0.003253,-0.725421,0.039938,-0.006304,-0.054584,-0.057299,0.066885,0.193064,-0.06319,-0.067103,-0.073003,-0.033187,-0.199447,-0.162793,-0.207939,-0.181785,0.19758
4,-0.030358,-0.914218,0.157586,0.169285,0.489985,-0.209388,-0.007162,-0.021926,-0.424394,-0.006511,-0.494002,-0.472024,-0.480570,-0.147931,-0.085259,-0.138378,-0.095478,-0.076871,-

In [5]:
y = pd.read_csv('/content/drive/My Drive/Dataset/skripsi/dos_dataset/y.csv')
y=y['Label']
y.head()

0    2
1    2
2    2
3    2
4    2
Name: Label, dtype: int64

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(normalized_df, y, test_size=0.3, random_state=0)
y_train = np.array(y_train)

y_train = np.array(y_train)
y_test = np.array(y_test)
X_train = np.array(X_train)
X_test = np.array(X_test)
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1]))

print('xtrain={}, ytrain={}, xtest={}, ytest={}'.format(X_train.shape,y_train.shape,X_test.shape,y_test.shape))

xtrain=(872058, 69), ytrain=(872058,), xtest=(373740, 69), ytest=(373740,)


In [7]:
!pip install xgboost

In [8]:
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(X_train, y_train)

import pickle
pickle.dump(model, open("/content/drive/My Drive/Dataset/skripsi/dos_dataset/final_xgb.pickle.dat", "wb"))

In [9]:
y_pred=model.predict(X_test)

In [11]:
y_train.shape

(872058,)

In [12]:
from sklearn import metrics
from sklearn.metrics import f1_score
print('XGBoost Classifier')

print('Accuracy = ', metrics.accuracy_score(y_test, y_pred)*100)
print("Confusion Matrix =\n", metrics.confusion_matrix(y_test, y_pred, labels=None, 
                                              sample_weight=None))
print("Recall =", metrics.recall_score(y_test, y_pred, labels=None, 
                                             pos_label=1, average='weighted', 
                                             sample_weight=None))
print("Classification Report =\n", metrics.classification_report(y_test, y_pred, 
                                                                 labels=None, 
                                                                 target_names=None, 
                                                                 sample_weight=None, 
                                                                 digits=2, 
                                                                 output_dict=False))

print("F1 Score = ",f1_score(y_test, y_pred, average='macro'))

XGBoost Classifier
Accuracy =  89.4456038957564
Confusion Matrix =
 [[17136     0     0     0     0     0     0     0]
 [    0 52978     2     0  6793     0     0     0]
 [    3     3 59642    42    13     3     0     3]
 [    1     2    23 60126    22     0   242     0]
 [    7 30340     6   111 25728     1     0    56]
 [    1     0     0     2     2 59764     2     0]
 [    1     1     0  1560     6     0 58493    21]
 [    1     2     0    18    29     0   127   427]]
Recall = 0.8944560389575641
Classification Report =
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     17136
           1       0.64      0.89      0.74     59773
           2       1.00      1.00      1.00     59709
           3       0.97      1.00      0.98     60416
           4       0.79      0.46      0.58     56249
           5       1.00      1.00      1.00     59771
           6       0.99      0.97      0.98     60082
           7       0.84      0.71    